# Nome: Raylander Marques Melo
# Matrícula: 586108

# Questão 1: Considere o conjunto de dados disponível em kc2.csv, organizado em 22 colunas, sendo as 21 primeiras colunas os atributos e a última coluna a saída. Os 21 atributos são referentes à caracterização de códigos-fontes para processamento de dados na NASA. A saída é a indicação de ausência (0) ou existência (1) de defeitos (os dados foram balanceados via subamostragem). Maiores detalhes sobre os dados podem ser conferidos em https://www.openml.org/search?type=data&sort=runs&id=1063&status=active.

In [118]:
!pip install numpy scikit-learn matplotlib pandas Jinja2

In [119]:
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from collections import defaultdict

In [120]:
data = np.genfromtxt('kc2.csv', delimiter=',')
x = data[:,:-1]
y = data[:,-1].astype(int)


In [121]:
def kfold(X, y, k, random_state=42):
    # Embaralha os dados
    indices = np.arange(X.shape[0])
    np.random.seed(random_state)
    np.random.shuffle(indices)
    
    # Divide os dados em k folds
    folds = np.array_split(indices, k)
    
    for i in range(k):
        # Cria os conjuntos de treino e teste
        test_indices = folds[i]
        train_indices = np.concatenate(folds[:i] + folds[i+1:])
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
        
        yield X_train, X_test, y_train, y_test

In [122]:
def normaliza(X):
    # Calculando a média e a distribuição 
    mean = X.sum(axis = 0) / X.shape[0]
    std = np.sqrt(((X-mean)**2).sum(axis=0)/(X.shape[0]-1))
    std[std == 0] = 1  # Evita divisão por zero
    # Normaliza os dados
    return ((X - mean) / std), mean, std

In [123]:
def inner_loop(x, y, model_class, grid, internal_kfold=10, scale_flag=True, verbose=True, random_state=12345):
    param_names = list(grid.keys())
    grid_search = np.meshgrid(*grid.values())
    grid_search = np.hstack([ np.atleast_2d(g.ravel()).T for g in grid_search ], dtype='object')
    
    for i, params in enumerate(grid_search):
        param_dict = dict(zip(param_names, params))

        if verbose:
            print(f"Testing parameters: {param_dict}")
        
        fold_scores = []
        
        for x_train, x_test, y_train, y_test in kfold(x, y, k=internal_kfold, random_state=random_state):
                        
            if scale_flag:
                x_train, mean, std = normaliza(x_train)
                x_test = (x_test - mean) / std
                model = model_class(**param_dict)
            else:
                model = model_class(**param_dict)
            
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test)
            fold_scores.append(accuracy_score(y_test, y_pred))
        
        yield param_dict, fold_scores

In [124]:
def run_nested_cv(x, y, model_class, grid, external_kfold=5, scale_flag=True, verbose=True, random_state=12345):
    param_names = list(grid.keys())
    grid_search = np.meshgrid(*grid.values())
    grid_search = np.hstack([ np.atleast_2d(g.ravel()).T for g in grid_search ], dtype='object')
    
    # Dicionário para armazenar métricas por combinação de parâmetros
    results = defaultdict(lambda: {'acc': [], 'rec': [], 'prec': [], 'f1': []})

    for x_train, x_test, y_train, y_test in kfold(x, y, k=external_kfold, random_state=random_state):
 
        for i, params in enumerate(grid_search):
            param_dict = dict(zip(param_names, params))
            param_key = str(param_dict)  # Chave para agrupar resultados

            # if verbose:
            #     print(f"[Fold {cont}] Testing parameters: {param_dict}")
            
            if scale_flag:
                x_train, mean, std = normaliza(x_train)
                x_test = (x_test - mean) / std
                model = model_class(**param_dict)
            else:
                model = model_class(**param_dict)
            
            model.fit(x_train, y_train)
            y_pred = model.predict(x_test)

            results[param_key]['acc'].append(accuracy_score(y_test, y_pred))
            results[param_key]['rec'].append(recall_score(y_test, y_pred, average='macro'))
            results[param_key]['prec'].append(precision_score(y_test, y_pred, average='macro'))
            results[param_key]['f1'].append(f1_score(y_test, y_pred, average='macro'))


    print(f"{'Parâmetros':<43} | {'Accuracy':^15} | {'Recall':^15} | {'Precision':^15} | {'F1-score':^15}")

    # Calcula média e desvio padrão das métricas por parâmetro
    avg_results = {}
    for param_key, metric_vals in results.items():
        avg_results[param_key] = {
            'mean_acc': np.mean(metric_vals['acc']),
            'std_acc': np.std(metric_vals['acc']),
            'mean_rec': np.mean(metric_vals['rec']),
            'std_rec': np.std(metric_vals['rec']),
            'mean_prec': np.mean(metric_vals['prec']),
            'std_prec': np.std(metric_vals['prec']),
            'mean_f1': np.mean(metric_vals['f1']),
            'std_f1': np.std(metric_vals['f1']),
        }

        if verbose:
            
            print('-' * 115)
            print(f"{param_key:<43} | "
                f"{avg_results[param_key]['mean_acc']:.4f} ± {avg_results[param_key]['std_acc']:.4f} | "
                f"{avg_results[param_key]['mean_rec']:.4f} ± {avg_results[param_key]['std_rec']:.4f} | "
                f"{avg_results[param_key]['mean_prec']:.4f} ± {avg_results[param_key]['std_prec']:.4f} | "
                f"{avg_results[param_key]['mean_f1']:.4f} ± {avg_results[param_key]['std_f1']:.4f}")
            
    print("\n")


                                 

## Questão 1 a) Considerando uma validação cruzada em 10 folds, avalie modelos de classificação binária nos dados em questão. Para tanto, use as abordagens abaixo

### – KNN (escolha k = 1 e k = 5, distância Euclidiana e Mahalonobis, totalizando 4 combinações);
### – Árvore de decisão (você pode usar uma implementação já existente, como a do scikit-learn, com índices de impureza de gini e entropia)

In [125]:
# Função para calcular distância Euclidiana
def euclidean_distance(p1, p2):
    soma = 0
    for i in range(len(p1)):
        soma += (p1[i] - p2[i]) ** 2
    return soma ** 0.5

In [126]:
# Função para calcular a distância de Mahalonobis
def mahalanobis_distance(p1, p2, cov_inv):
    diff = p1 - p2
    return np.sqrt(np.dot(np.dot(diff, cov_inv), diff.T))

In [127]:
# Função para encontrar os K vizinhos mais próximos
def get_neighbors(train_data, train_labels, test_point, k, metric='euclidean'):
    distancias = []
    for i in range(len(train_data)):
        if metric == 'mahalanobis':
            cov_inv = np.linalg.inv(np.cov(train_data.T))
            distancia = mahalanobis_distance(test_point, train_data[i], cov_inv)
        elif metric == 'euclidean':
            distancia = euclidean_distance(test_point, train_data[i])
        distancias.append((distancia, train_labels[i]))
    
    distancias.sort()  # Ordena pela menor distância
    vizinhos = distancias[:k]
    return [label for (_, label) in vizinhos]

In [128]:
# Função para determinar a classe mais comum entre os vizinhos
def vote(vizinhos):
    contagem = {}
    for label in vizinhos:
        contagem[label] = contagem.get(label, 0) + 1
    return max(contagem, key=contagem.get)

In [129]:
# Função principal do KNN
class KNN:
    def __init__(self, k=2, metric='euclidean'):
        self.k = k
        self.metric = metric

    def fit(self, X, y):
        self.train_data = X
        self.train_labels = y

    def predict(self, X):
        return knn_predict(self.train_data, self.train_labels, X, self.k, self.metric)
    
def knn_predict(train_data, train_labels, test_data, k=2, metric='euclidean'):
    predicoes = []
    for ponto in test_data:
        vizinhos = get_neighbors(train_data, train_labels, ponto, k,metric=metric)
        classe = vote(vizinhos)
        predicoes.append(classe)
    return predicoes

## Questão 1 b) Para cada modelo criado, reporte valor médio e desvio padrão das métricas de acurácia, revocação, precisão e F1-score.

In [131]:
# Executando o KNN com os dados de treino e teste
external_kfold = 10

methods_summary = {'KNN': {'class': KNN, 'scale': True},
                   'DT' : {'class': DecisionTreeClassifier, 'scale': False}}

# KNN
methods_summary['KNN']['grid'] = {'k': [1, 5],         # k - número de vizinhos
                                  'metric': ['euclidean','mahalanobis']}        # metric            
# Decision Tree
methods_summary['DT']['grid'] = {'criterion': ['gini', 'entropy'],                # criterion
                                 'max_depth': [None]} # max_depth

trained_models = {}
for method, info in methods_summary.items():
    print(f"\n                                                      {method}\n")
    run_nested_cv(x=x, y=y, model_class=info['class'],
                                        grid=info['grid'], scale_flag=info['scale'],
                                        external_kfold=external_kfold, verbose=True, random_state=42)
    



                                                      KNN

Parâmetros                                  |    Accuracy     |     Recall      |    Precision    |    F1-score    
-------------------------------------------------------------------------------------------------------------------
{'k': 1, 'metric': 'euclidean'}             | 0.7578 ± 0.1134 | 0.7613 ± 0.1124 | 0.7655 ± 0.1184 | 0.7541 ± 0.1129
-------------------------------------------------------------------------------------------------------------------
{'k': 5, 'metric': 'euclidean'}             | 0.7903 ± 0.1194 | 0.7945 ± 0.1188 | 0.7983 ± 0.1203 | 0.7871 ± 0.1187
-------------------------------------------------------------------------------------------------------------------
{'k': 1, 'metric': 'mahalanobis'}           | 0.7108 ± 0.0922 | 0.7121 ± 0.0900 | 0.7222 ± 0.0912 | 0.7029 ± 0.0921
-------------------------------------------------------------------------------------------------------------------
{'k': 5, 'me